# Patch-based Change Detection with Siamese Networks

By Zhenchao Zhang (z.zhang-1@utwente.nl)

## Imports
All the imports are defined here

In [1]:
from __future__ import print_function
%matplotlib inline

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from random import choice, sample
from PIL import Image

import PIL.ImageOps 
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os
import linecache
import csv
import time
import math
# import cv2

#os.chdir('/home/zhangz1/Data/CodeVE/ChangeDetection/')

cwd = os.getcwd()
print(cwd)

/home/zhenchao/CodePyTor


In [2]:
cwd = os.getcwd()
print(cwd)

/home/zhenchao/CodePyTor


## Define data Loader

In [5]:
def batch_loader(Pos_ids,Neg_ids,linesPos,linesNeg, TrainFlag):  # Loader one batch. Input: 2 lists.   Output: a batch
    num = len(Pos_ids)  # should be 128
       
    for i in range(0,num):
        line = linesPos[Pos_ids[i]]
        
        line.strip('\n')
        img_list= line.split()  # img_list has 4 elements
        
        img0 = Image.open(cwd+'/Patches/'+img_list[0])  #  ALS, 1 channel, float
        img1 = Image.open(cwd+'/Patches/'+img_list[1])  #  DIM, 1 channel, float
        img2 = Image.open(cwd+'/Patches/'+img_list[2])  #  Ortho, 3 channels, int [0,255]
        
        # img0 = img0.resize((112,112), resample=PIL.Image.BICUBIC)
        # img1 = img1.resize((112,112), resample=PIL.Image.BICUBIC)
        # img2 = img2.resize((112,112), resample=PIL.Image.BICUBIC)
        
        t = ToTensor()(img0)  # 100*100
        t0 = (t-Hmin)/Hdel
        t0 = t0.unsqueeze(0)  # unsqueeze to add artificial first dimension
        
        t = ToTensor()(img1)  # 100*100
        t1 = (t-Hmin)/Hdel
        t1 = t1.unsqueeze(0)
        
        t2 = ToTensor()(img2)  # After: 3*100*100
        t2 = t2.unsqueeze(0)   # Check if all t0, t1 and t2 are in [0,1]
        
        label = float(img_list[3])
        label = torch.tensor(label)
        label = label.unsqueeze(0).unsqueeze(0)
        
        if i==0:
            T0 = t0
            T1 = t1
            T2 = t2
            T3 = label
        else:
            T0 = torch.cat([T0,t0],0)
            T1 = torch.cat([T1,t1],0)
            T2 = torch.cat([T2,t2],0)
            T3 = torch.cat([T3,label],0)
            
    batch = [T0, T1, T2, T3]
    return batch

## Define network

In [ ]:
class CDNet(nn.Module):    # [SI-HH-conc-SW]   SI-CNN (HH) (Shared weights)(middle stack fusion)
    def __init__(self):
        super(CDNet, self).__init__()
        
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 3, kernel_size=5)  # 3 or 5? 
        self.pool = nn.MaxPool2d(2, 2)
        
        # after concatenation
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.conv3 = nn.Conv2d(10, 16, 5)
        
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)   # 2-D outputs
        
    def forward(self,X0,X1,X2):
        
        X0 = self.pool(F.relu(self.conv1(X0)))     # ALS data, 3 chann, 48*48
        X1 = self.pool(F.relu(self.conv1(X1)))     # DIM data, 3 chann, 48*48
        
        x = torch.cat([X0,X1],1)  # size: [64,6,48,48]
        
        x = self.pool(F.relu(self.conv2(x)))  # size: [64,10,22,22]
        x = self.pool(F.relu(self.conv3(x)))  # size: [64,16,9,9]
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
class CDNet(nn.Module):    # [SI-HH-conc-DW] SI-CNN (HH) (UNshared weights)(middle stack fusion)
    def __init__(self):
        super(CDNet, self).__init__()
               
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 3, kernel_size=5)  # 3 or 5? 
        self.conv1A = nn.Conv2d(1, 3, kernel_size=5)  # 3 or 5? 
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # after concatenation
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.conv3 = nn.Conv2d(10, 16, 5)
             
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)   # 2-D outputs
        
    def forward(self,X0,X1,X2):
        
        X0 = self.pool(F.relu(self.conv1(X0)))     # ALS data, 3 chann, 48*48
        X1 = self.pool(F.relu(self.conv1A(X1)))     # DIM data, 3 chann, 48*48
        
        x = torch.cat([X0,X1],1)  # size: [64,6,48,48]
        
        x = self.pool(F.relu(self.conv2(x)))  # size: [64,10,22,22]
        x = self.pool(F.relu(self.conv3(x)))  # size: [64,16,9,9]
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
class CDNet(nn.Module):    # [SI-HH-diff-SW]  SI-CNN (HH) (Shared weights)(End abs minus fusion)
    def __init__(self):
        super(CDNet, self).__init__()
        
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.conv3 = nn.Conv2d(10, 16, 5)
        
        # after concatenation
        
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)   # 2-D outputs
        self.fc3 = nn.Linear(84, 2)   # 2-D outputs
        
    def forward(self,X0,X1,X2):
        res = []
        
        for i in range(2): # Siamese nets; sharing weights
            if i==0:
                x=X0
            if i==1:
                x=X1
            
            x = self.pool(F.relu(self.conv1(x)))     # size: [64,6,48,48]
            x = self.pool(F.relu(self.conv2(x)))     # size: [64,10,22,22]
            x = self.pool(F.relu(self.conv3(x)))     # size: [64,10,22,22]
            
            x = x.view(-1, 16 * 9 * 9)    # size: [64,10,22,22]
            x = self.fc1(x)     # size: [120*1]
            x = F.relu(x)
            res.append(x)

        res = torch.abs(res[1] - res[0])   # abs|A-B|  # size: [120*1]
        res = F.relu(self.fc2(res))
        res = self.fc3(res)   # size: [2*1]
        return res
       

In [ ]:
class CDNet(nn.Module):    # SI-CNN (HH) (UNshared weights)(End abs minus fusion)
    def __init__(self):
        super(CDNet, self).__init__()
               
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)   # 1st branch
        self.conv2 = nn.Conv2d(6, 10, 5)
        
        self.conv1A = nn.Conv2d(1, 6, kernel_size=5)  # 2nd branch
        self.conv2A = nn.Conv2d(6, 10, 5)
        
        self.pool = nn.MaxPool2d(2, 2)
           
        self.fc1 = nn.Linear(10 * 22 * 22, 120)
        self.fc1A = nn.Linear(10 * 22 * 22, 120)
        
        self.fc2 = nn.Linear(120, 2)   # 2-D outputs
        
    def forward(self,X0,X1,X2):
        res = []
        
        for i in range(2): # Siamese nets; UNsharing weights
            if i==0:
                x = X0
                x = self.pool(F.relu(self.conv1(x)))     # size: [64,6,48,48]
                x = self.pool(F.relu(self.conv2(x)))     # size: [64,10,22,22]
                x = x.view(x.shape[0], -1)    # size: [64,10,22,22]
                x = self.fc1(x)     # size: [120*1]
                x = F.relu(x)
                res.append(x)
                
            if i==1:
                x = X1
                x = self.pool(F.relu(self.conv1A(x)))     # size: [64,6,48,48]
                x = self.pool(F.relu(self.conv2A(x)))     # size: [64,10,22,22]
                x = x.view(x.shape[0], -1)    # size: [64,10,22,22]
                x = self.fc1A(x)     # size: [120*1]
                x = F.relu(x)
                res.append(x)

        res = torch.abs(res[1] - res[0])   # abs|A-B|  # size: [120*1]
        dist = self.fc2(res)   # size: [2*1]
        return dist   # size: [2*1]


In [ ]:
class CDNet(nn.Module):    # [SI-HHC-conc-DW][filter=5][NO Batch-Normalization] SI-CNN (HH + color!) (UNshared weights)(middle stack fusion)
    def __init__(self):
        super(CDNet, self).__init__()
               
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)  # 3 or 5? 
        self.conv1A = nn.Conv2d(4, 6, kernel_size=5)  # 3 or 5?
               
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.conv2A = nn.Conv2d(6, 10, 5)
        
        self.conv3 = nn.Conv2d(10, 12, 5)
        self.conv3A = nn.Conv2d(10, 12, 5)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # after concatenation
        self.conv4 = nn.Conv2d(24, 30, 5)
        
        self.fc1 = nn.Linear(30 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)   # 2-D outputs
        
    def forward(self,X0,X1,X2):
        
        x0 = self.pool(F.relu(self.conv1(X0)))     # ALS data
        x0 = self.pool(F.relu(self.conv2(x0)))     # 10*22*22
        x0 = F.relu(self.conv3(x0))                # 12*18*18
        
        x1 = torch.cat([X1,X2],1)  # Concatenate DSM2 and ortho
        x1 = self.pool(F.relu(self.conv1A(x1)))     # DIM data
        x1 = self.pool(F.relu(self.conv2A(x1)))     # 10*22*22
        x1 = F.relu(self.conv3A(x1))                # 12*18*18
        
        x = torch.cat([x0,x1],1)  # size: [64,24,18,18]
        x = self.pool(F.relu(self.conv4(x)))   # [64,30,7,7]
        
        x = x.view(-1, 30 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
class CDNet(nn.Module):    # [SI-HHC-conc-DW] [filter=3][8 Conv bocks before] SI-CNN (HH + color!) (UNshared weights)(middle stack fusion)
    def __init__(self):
        super(CDNet, self).__init__()
        
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,stride=1,padding=1)  # 3 or 5? 
        self.conv1A = nn.Conv2d(4, 32, kernel_size=3,stride=1,padding=1)  # 3 or 5?
                       
        self.conv2 = nn.Conv2d(32, 32, 3, 1, 1)
        self.conv2A = nn.Conv2d(32, 32, 3, 1, 1)
                
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv3A = nn.Conv2d(32, 64, 3, 1, 1)
        
        self.conv4 = nn.Conv2d(64, 64, 3, 1, 1)
        self.conv4A = nn.Conv2d(64, 64, 3, 1, 1)
        
        self.conv5 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv5A = nn.Conv2d(64, 128, 3, 1, 1)
        
        self.conv6 = nn.Conv2d(128, 128, 3, 1, 1)
        self.conv6A = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.conv7 = nn.Conv2d(128, 128, 3, 1, 1)
        self.conv7A = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.conv8 = nn.Conv2d(128, 128, 3, 1, 1)
        self.conv8A = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.pool = nn.MaxPool2d(2, 2, padding=0)
        
        # after concatenation
        self.conv9 = nn.Conv2d(256, 128, 1, 1, 0)
        self.conv10 = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.fc1 = nn.Linear(128 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 2)  # 2-D outputs
        
    def forward(self,X0,X1,X2):
        # ALS patch
        x0 = F.relu(self.conv1(X0))     
        x0 = self.pool(F.relu(self.conv2(x0)))
        
        x0 = F.relu(self.conv3(x0))     
        x0 = self.pool(F.relu(self.conv4(x0)))
        
        x0 = F.relu(self.conv5(x0))     
        x0 = self.pool(F.relu(self.conv6(x0)))
        
        x0 = F.relu(self.conv7(x0))     
        x0 = F.relu(self.conv8(x0))
        
        # DSM2 and ortho               
        x1 = torch.cat([X1,X2],1)  # Concatenate 
        x1 = F.relu(self.conv1A(x1))     
        x1 = self.pool(F.relu(self.conv2A(x1)))
        
        x1 = F.relu(self.conv3A(x1))     
        x1 = self.pool(F.relu(self.conv4A(x1)))
        
        x1 = F.relu(self.conv5A(x1))     
        x1 = self.pool(F.relu(self.conv6A(x1)))
        
        x1 = F.relu(self.conv7A(x1))     
        x1 = F.relu(self.conv8A(x1))
                
        # Concatenate
        x = torch.cat([x0,x1],1)  # size: [64,256,14,14]
        x = F.relu(self.conv9(x))
        x = self.pool(F.relu(self.conv10(x)))
        
        x = x.view(-1, 128 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [ ]:
class CDNet(nn.Module):    # [SI-HHC-conc-DW] [filter==3][4 Conv bocks before] SI-CNN (HH + color!) (UNshared weights)(middle stack fusion)
    def __init__(self):
        super(CDNet, self).__init__()
        
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,stride=1,padding=1)  # 3 or 5? 
        self.conv1A = nn.Conv2d(4, 32, kernel_size=3,stride=1,padding=1)  # 3 or 5?
                       
        #self.conv2 = nn.Conv2d(32, 32, 3, 1, 1)
        #self.conv2A = nn.Conv2d(32, 32, 3, 1, 1)
                
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv3A = nn.Conv2d(32, 64, 3, 1, 1)
        
        #self.conv4 = nn.Conv2d(64, 64, 3, 1, 1)
        #self.conv4A = nn.Conv2d(64, 64, 3, 1, 1)
        
        self.conv5 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv5A = nn.Conv2d(64, 128, 3, 1, 1)
        
        #self.conv6 = nn.Conv2d(128, 128, 3, 1, 1)
        #self.conv6A = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.conv7 = nn.Conv2d(128, 128, 3, 1, 1)
        self.conv7A = nn.Conv2d(128, 128, 3, 1, 1)
        
        #self.conv8 = nn.Conv2d(128, 128, 3, 1, 1)
        #self.conv8A = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.pool = nn.MaxPool2d(2, 2, padding=0)
        
        # after concatenation
        self.conv9 = nn.Conv2d(256, 128, 1, 1, 0)
        self.conv10 = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.fc1 = nn.Linear(128 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 2)  # 2-D outputs
        
    def forward(self,X0,X1,X2):
        # ALS patch
        x0 = self.pool(F.relu(self.conv1(X0)))     
        # x0 = self.pool(F.relu(self.conv2(x0)))
        
        x0 = self.pool(F.relu(self.conv3(x0)))     
        # x0 = self.pool(F.relu(self.conv4(x0)))
        
        x0 = self.pool(F.relu(self.conv5(x0)))     
        #x0 = self.pool(F.relu(self.conv6(x0)))
        
        x0 = F.relu(self.conv7(x0))     
        #x0 = F.relu(self.conv8(x0))
        
        # DSM2 and ortho               
        x1 = torch.cat([X1,X2],1)  # Concatenate 
        x1 = self.pool(F.relu(self.conv1A(x1)))     
        #x1 = self.pool(F.relu(self.conv2A(x1)))
        
        x1 = self.pool(F.relu(self.conv3A(x1)))     
        #x1 = self.pool(F.relu(self.conv4A(x1)))
        
        x1 = self.pool(F.relu(self.conv5A(x1)))     
        #x1 = self.pool(F.relu(self.conv6A(x1)))
        
        x1 = F.relu(self.conv7A(x1))     
        #x1 = F.relu(self.conv8A(x1))
                
        # Concatenate
        x = torch.cat([x0,x1],1)  # size: [64,256,14,14]
        x = F.relu(self.conv9(x))
        x = self.pool(F.relu(self.conv10(x)))
        
        x = x.view(-1, 128 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [13]:
class CDNet(nn.Module):    # [SI-HHC-conc-DW] [filter=5&3][3 Conv bocks before] SI-CNN (HH + color!) (UNshared weights)(middle stack fusion)
    def __init__(self):
        super(CDNet, self).__init__()
        
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        
        # Convolution Layer 1      # 1x100x100 input
        self.conv1 = nn.Conv2d(1, 6,  kernel_size=5,stride=1,padding=0)  # 3 or 5? 
        self.conv1A = nn.Conv2d(4, 6, kernel_size=5,stride=1,padding=0)  # 3 or 5?
                       
        #self.conv2 = nn.Conv2d(32, 32, 3, 1, 1)
        #self.conv2A = nn.Conv2d(32, 32, 3, 1, 1)
                
        self.conv3 = nn.Conv2d(6, 8, 3, 1, 1)
        self.conv3A = nn.Conv2d(6, 8, 3, 1, 1)
        
        #self.conv4 = nn.Conv2d(64, 64, 3, 1, 1)
        #self.conv4A = nn.Conv2d(64, 64, 3, 1, 1)
        
        self.conv5 = nn.Conv2d(8, 10, 3, 1, 1)
        self.conv5A = nn.Conv2d(8, 10, 3, 1, 1)
        
        #self.conv6 = nn.Conv2d(128, 128, 3, 1, 1)
        #self.conv6A = nn.Conv2d(128, 128, 3, 1, 1)
        
        # self.conv7 = nn.Conv2d(10, 12, 3, 1, 1)
        # self.conv7A = nn.Conv2d(10, 12, 3, 1, 1)
        
        #self.conv8 = nn.Conv2d(128, 128, 3, 1, 1)
        #self.conv8A = nn.Conv2d(128, 128, 3, 1, 1)
        
        self.pool = nn.MaxPool2d(2, 2, padding=0)
        
        # after concatenation
        self.conv9 = nn.Conv2d(20, 24, 3, 1, 1)
        # self.conv10 = nn.Conv2d(30, 32, 3, 1, 1)
        
        self.fc1 = nn.Linear(24 * 24 * 24, 128)
        self.fc2 = nn.Linear(128, 2)  # 2-D outputs
        
    def forward(self,X0,X1,X2):
        # ALS patch
        x0 = self.pool(F.relu(self.conv1(X0)))     
        # x0 = self.pool(F.relu(self.conv2(x0)))
        
        x0 = self.pool(F.relu(self.conv3(x0)))     
        # x0 = self.pool(F.relu(self.conv4(x0)))
        
        #x0 = self.pool(F.relu(self.conv5(x0)))     
        #x0 = self.pool(F.relu(self.conv6(x0)))
        
        x0 = F.relu(self.conv5(x0))     
        #x0 = F.relu(self.conv8(x0))
        
        # DSM2 and ortho               
        x1 = torch.cat([X1,X2],1)  # Concatenate 
        x1 = self.pool(F.relu(self.conv1A(x1)))     
        #x1 = self.pool(F.relu(self.conv2A(x1)))
        
        x1 = self.pool(F.relu(self.conv3A(x1)))     
        #x1 = self.pool(F.relu(self.conv4A(x1)))
        
        #x1 = self.pool(F.relu(self.conv5A(x1)))     
        #x1 = self.pool(F.relu(self.conv6A(x1)))
        
        x1 = F.relu(self.conv5A(x1))     
        #x1 = F.relu(self.conv8A(x1))
                
        # Concatenate
        x = torch.cat([x0,x1],1)  # size: [64,256,14,14]
        x = F.relu(self.conv9(x))
        #x = self.pool(F.relu(self.conv9(x)))
        
        x = x.view(-1, 24 * 24 * 24)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [6]:
class CDNet(nn.Module):    # [FF-HH]
    def __init__(self):
        super(CDNet, self).__init__()
        
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        # which is what we are using.
        
        # Convolution Layer 1                             # 28 x 28 x 1  (input)
        self.conv1 = nn.Conv2d(2, 6, kernel_size=5)      # 24 x 24 x 20  (after 1st convolution)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 22 * 22, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self,X0,X1,X2):
        
        x = torch.cat([X0,X1],1)  # size of d: [64,2,100,100]
        x = self.pool(F.relu(self.conv1(x)))     # 20 x 20 x 30  (after 2nd convolution)
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 22 * 22)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
class CDNet(nn.Module):   # [FF-HHC],   Color
    def __init__(self):
        super(CDNet, self).__init__()
        
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        # which is what we are using.
        
        # Convolution Layer 1                             # 28 x 28 x 1  (input)
        self.conv1 = nn.Conv2d(5, 8, kernel_size=5)      # 24 x 24 x 20  (after 1st convolution)
        self.conv2 = nn.Conv2d(8, 10, 5)  # 6->10?
        self.conv3 = nn.Conv2d(10, 16, 3, 1, 1)  # 6->10?
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(16 * 22 * 22, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self,X0,X1,X2):
        
        x = torch.cat([X0,X1,X2],1);  # size of d: [64,5,100,100]
        
        x = self.pool(F.relu(self.conv1(x)))     # 20 x 20 x 30  (after 2nd convolution)
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        x = x.view(-1, 16 * 22 * 22)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [8]:
net4 = CDNet().cuda()
net = net4
net.train()

weights = [1.00, 5.18]   # default: [1.0, 36.94], [1.0, 10.0]
class_weights = torch.FloatTensor(weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights,size_average=True)  # BCEWithLogitsLoss()  #   True: loss is averaged over each loss element in batch

optimizer = optim.SGD(net.parameters(), lr=0.008, momentum=0.9)  # Good: SGD, 0.01
#optimizer = optim.Adamax(net.parameters(), lr=0.003, betas=(0.9, 0.999))  
max_train_acc = 0.0;    # initializxe only once, when defining the empty model
max_val_acc = 0.0;
max_prec = 0.0;

train_losses = []
train_accuracys = []
val_losses = []
val_accuracys = []

In [ ]:
print(net)

## Training
The top row and the bottom row of any column is one pair. The 0s and 1s correspond to the column of the image.
0 indiciates dissimilar, and 1 indicates similar.

In [9]:
# Adjust the learning rate
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)  # 0.003, 0.001,0.0005


In [ ]:
# net = torch.load('Saved_Model_test.pth')
net = CDNet().cuda()
net.load_state_dict(torch.load('/home/zhangz1/Data/CodeVE/ChangeDetection/Saved_Model_max_val_prec(0.4654).pt'))
net.train()

In [17]:
# Define the validation function.  Input: .   Output: loss per iter and accuracy
def Validation(linesVal,NumVal,net):

    val_losses = 0.0
    val_iterations = 0
    val_correct = 0
    val_total = 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0

    net.eval()   # Put the network into evaluate mode
    TrainFlag = 0
    count = 0

    Pos_ids = []
    Neg_ids = []

    for i in range(0,NumVal): # In each iterarion, select at most 64 samples
        if i%20000==0:  # Flag
            print ('Val-> Iter #: %d/%d' % (i,NumVal))
        Pos_ids.append(i)
        count = count+1        
        if count == 128 or i==NumVal-1:  # In validation mode, Neg_ids and linesNeg don't matter.
            batch = batch_loader(Pos_ids,Neg_ids,linesVal,linesVal,TrainFlag);  #  A batch is a 4-element list
            # Until now, we get a batch (64D OR less)
            # Now start validation:
            IM0, IM1, IM2, Lbs = batch
            IM0, IM1, IM2, Lbs = Variable(IM0).cuda(), Variable(IM1).cuda(), Variable(IM2).cuda(), Variable(Lbs).cuda()

            # forward
            outputs = net(IM0,IM1,IM2)
            Lbs = torch.tensor(Lbs, dtype=torch.int64)  # convert from float to LongTensor
            Lbs = Lbs.view(count)
            Lbs = Variable(Lbs).cuda()

            # Record the correct predictions for validation data             
            loss = criterion(outputs, Lbs)  # average loss
            val_losses += loss.data[0] # Accumulate the loss

            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == Lbs.data).sum()
            val_total += Lbs.size(0)
            val_iterations += 1
            
            # Calculate TP, TN, FP, FN
            A = predicted + Lbs.data
            B = predicted - Lbs.data
            C = predicted * Lbs.data
            
            tp = A.eq(2).sum().item()
            fn = B.eq(-1).sum().item()
            fp = B.eq(1).sum().item()
            tn = Lbs.size(0)-tp-fn-fp
            TP = TP+tp
            FN = FN+fn
            FP = FP+fp
            TN = TN+tn
            
            count = 0  # empty it
            Pos_ids = []  # empty it
            
            # if val_iterations%50==0:
            #     print ('Val-> Iter #: %d' % val_iterations)

    # Record the val_loss & val_accuracy for all the validation samples
    # loss per mini-batch, recorded after every epoch
    prec = -1.0
    reca = -1.0
    if (TP+FP)!=0:
        prec = TP/(TP+FP)
    if (TP+FN)!=0:
        reca = TP/(TP+FN)

    return val_losses/val_iterations, 100*val_correct.item()/val_total, val_total, TP,TN,FP,FN, prec, reca

In [ ]:
# Start training
num_epochs = 20   # default 20
batch_size = 128
sample_size = 138459
num_iters = np.int16(sample_size/batch_size)   # 1081.7
print(num_iters)
# num_iters = 1  # for debugging

Hmin=-16.64   # minimum height in the block
Hmax=160.51
Hdel=Hmax-Hmin       # Height ranges  

fpTrain = open("train-merge(138459).txt", "r")
linesTrain = fpTrain.readlines()
fpTrain.close()

# Generate two flag vectors: Pos:[0,20000)   Neg:[0,20000)

fpVal = open("SampName_Val_(left+right)(107036)(noXY)_refined.txt", "r")    # Validation file
linesVal = fpVal.readlines()
fpVal.close()
NumVal = len(linesVal)

iter_loss = 0.0
iterations = 0
train_correct = 0
train_total = 0
net.train()    # Put the network into training mode
TrainFlag = 1

linesEmpty = 0

In [ ]:
num_epochs = 20

start_time = time.time()
print(num_epochs)

for epoch in range(0,num_epochs):  
    if epoch<10:
        optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9)  # 0.003, 0.001,0.0005
    if epoch>=10:
        optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9)  # 0.003, 0.001,0.0005

    # In each iterarion, select half batches from positive samples, half from neg
    Flags = np.random.permutation(sample_size);
    
    iter_loss = 0.0  # Initialize at the beginning of one epoch
    iterations = 0  # Number of iters
    train_correct = 0
    train_total = 0
    
    for i in range(0,num_iters):  # num_iters=1081
        if i%50==0:  # Initialization per 100 iterations
            print ('Epoch %d/%d: Train-> Iter #: %d/%d' % (epoch+1, num_epochs, i,num_iters))
        
        net.train()    # Put the network into training mode
        TrainFlag = 1
        
        Pos_ids = []
        Neg_ids = []
        
        start = np.int64(i*batch_size)  # batch_size = 128
        end   = np.int64((i+1)*batch_size)
        
        for index in range(start,end):  # Line index
            Pos_ids.append(Flags[index])
        
        batch = batch_loader(Pos_ids,Neg_ids,linesTrain,linesEmpty,TrainFlag);  #  A batch is a 4-element list
        
        # Now start training with one batch
        IM0, IM1, IM2, Lbs = batch
        IM0, IM1, IM2, Lbs = Variable(IM0).cuda(), Variable(IM1).cuda(), Variable(IM2).cuda(), Variable(Lbs).cuda()
        
        # forward + backward + optimize
        optimizer.zero_grad()    # zero the parameter gradients
        outputs = net(IM0,IM1,IM2)
        Lbs = torch.tensor(Lbs, dtype=torch.int64)  # convert from float to LongTensor
        Lbs =  Lbs.view(128)
        Lbs = Variable(Lbs).cuda()
        
        loss = criterion(outputs, Lbs)
        iter_loss += loss.data[0] # Accumulate the loss
        loss.backward()
        optimizer.step()
        
        # Record the correct predictions for training data 
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == Lbs.data).sum()
        iterations += 1
        train_total += Lbs.size(0)
    
    # Record the training results, after one epoch
    train_losses.append(iter_loss/iterations)  # loss per iteration
    train_accuracys.append((100 * train_correct.item() / train_total))

    ############################
    # Validate - Calculate the Val loss&accuracy per epoch 

    
    if epoch%3 !=0 and epoch!=num_epochs-1:
        continue
    
    val_los,val_accur,val_total,TP,TN,FP,FN, prec,reca = Validation(linesVal,NumVal,net)
    
    val_losses.append(val_los)  # loss on all the val set
    val_accuracys.append(val_accur)
    
    print ('Tr Loss: %.4f, Tr Acc: %.4f,  Val Loss: %.4f, Val Acc: %.4f'
           %(train_losses[-1], train_accuracys[-1], val_losses[-1], val_accuracys[-1]))
    print ('TP:%4d,  TN:%4d,  FP:%4d,  FN:%4d,  prec: %.4f, reca: %.4f\n'
               %(TP,TN,FP,FN, prec, reca))

    # After 100 iters, check if this is good model, save it
    if train_accuracys[-1] > max_train_acc:
        #torch.save(net, 'Saved_Model_max_train_acc.pth');
        torch.save(net.state_dict(), 'Saved_Model_max_train_acc.pt');
        max_train_acc = train_accuracys[-1];
        
    if val_accuracys[-1] > max_val_acc:
        #torch.save(net, 'Saved_Model_max_val_acc.pth');
        torch.save(net.state_dict(), 'Saved_Model_max_val_acc.pt');
        max_val_acc = val_accuracys[-1];
        
    if prec > max_prec:
        #torch.save(net, 'Saved_Model_max_val_prec.pth');
        torch.save(net.state_dict(), 'Saved_Model_max_val_prec.pt');
        max_prec = prec;    

print('\nFinished Training and validation.')
print('max_train_acc: %.4f, max_val_acc: %.4f, max_val_prec: %.4f\n'%(max_train_acc,max_val_acc,max_prec));

t = (time.time()-start_time)/60.0
print("--- %s min ---\n" % t)

print(len(train_losses))
print(len(val_losses))
print(train_losses)
print(val_losses)

In [ ]:
print(len(train_losses))
print(len(val_losses))
print(train_losses)
print(val_losses)

In [ ]:
# Save current model & the best model.
torch.save(net, 'Saved_Model(T99.9)(V97.0).pth')  # net.state_dict()
torch.save(net.state_dict(), 'Saved_Model(T99.9)(V97.0).pt')

In [ ]:
print(max_train_acc,max_val_acc,max_prec)

## Plot & save

In [ ]:
cnt = len(train_losses)
#cnt = len(train_accuracys)

font = {'family' : 'normal',
        'size'   : 15} # 'weight' : 'bold',

plt.rc('font', **font)
plt.rc('axes', linewidth=1)

plt.plot(range(0, cnt), train_losses[0:cnt], 'b', label="train_loss",linewidth=2) # plotting t, a separately 
plt.legend(bbox_to_anchor=(1.2, 0.5), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
#cnt = len(train_losses)
cnt = len(train_accuracys)

font = {'family' : 'normal',
        'size'   : 15} # 'weight' : 'bold',

plt.rc('font', **font)
plt.rc('axes', linewidth=1)

plt.plot(range(0, cnt), train_accuracys[0:cnt], 'b', label="train_loss",linewidth=2) # plotting t, a separately 
plt.legend(bbox_to_anchor=(1.2, 0.5), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
print(len(train_losses))
print(len(train_accuracys))
print(len(val_losses))
print(len(val_accuracys))

# Save the list to txt:  # Validation result: 1-TP, 2-FN, 3-FP, 4-TN
with open('Model_train_losses(Siam).txt', 'w') as f:
    for item in train_losses:
        f.write("%f\n" % item)
f.close()

with open('Model_train_accuracys(Siam).txt', 'w') as f:
    for item in train_accuracys:
        f.write("%f\n" % item)
f.close()

with open('Model_val_losses(Siam).txt', 'w') as f:
    for item in val_losses:
        f.write("%f\n" % item)
f.close()

with open('Model_val_accuracys(Siam).txt', 'w') as f:
    for item in val_accuracys:
        f.write("%f\n" % item)

f.close()

## Testing

In [ ]:
start_time = time.time()

Pos_ids = []  # empty it
Neg_ids = []
count = 0
train_correct = 0
outs = []  # Result: 1-TP, 2-FN, 3-FP, 4-TN
prec=0
reca=0

os.chdir('/home/zhangz1/Data/CodeVE/ChangeDetection/Test')
cwd = os.getcwd()
print(cwd)

fpVal = open("SampName_All4Blocks_padded(noXY)(135828).txt", "r")    # Validation file
linesVal = fpVal.readlines()
fpVal.close()
NumVal = len(linesVal)
print(NumVal)

TP = 0
TN = 0
FP = 0
FN = 0

for i in range(0,NumVal): # In each iterarion, select at most 64 samples
    if i%5000==0:
        print(i)
    Pos_ids.append(i)
    count = count+1        
    if count == 128 or i== NumVal-1:  # In validation mode, Neg_ids and linesNeg don't matter.
        # Neg_ids and linesNeg don't matter.
        TrainFlag = 0
        
        batch = batch_loader(Pos_ids,Neg_ids,linesVal,linesVal,TrainFlag);  #  A batch is a 4-element list
        
        IM0, IM1, IM2, Lbs = batch
        IM0, IM1, IM2, Lbs = Variable(IM0).cuda(), Variable(IM1).cuda(), Variable(IM2).cuda(), Variable(Lbs).cuda()
        
        # forward
        outputs = net(IM0,IM1,IM2)
        Lbs = torch.tensor(Lbs, dtype=torch.int64)  # convert from float to LongTensor
        Lbs = Lbs.view(count)
        Lbs = Variable(Lbs).cuda()
        
        # Record the correct predictions             
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == Lbs.data).sum()

        for j in range(0,count):
            if predicted[j].item()==1 and Lbs.data[j].item()==1:
                outs.append(1)
                TP = TP+1
            if predicted[j].item()==0 and Lbs.data[j].item()==1:
                outs.append(2)
                FN = FN+1
            if predicted[j].item()==1 and Lbs.data[j].item()==0:
                outs.append(3)
                FP = FP+1
            if predicted[j].item()==0 and Lbs.data[j].item()==0:
                outs.append(4)
                TN = TN+1
    
        Pos_ids = []  # empty it
        count = 0

print(train_correct)
print(len(outs))

if (TP+FP)!=0:
    prec = TP/(TP+FP)
if (TP+FN)!=0:
    reca = TP/(TP+FN)
F1 = 2*prec*reca/(prec+reca)
    
print ('TP:%4d,  TN:%4d,  FP:%4d,  FN:%4d,  prec: %.4f, reca: %.4f, F1: %.4f\n'
                   %(TP,TN,FP,FN, prec, reca, F1))

t = (time.time()-start_time)/60.0
print("--- %s min ---\n" % t)

In [ ]:
print(len(outs))

# Save the list to txt:  # Validation result: 1-TP, 2-FN, 3-FP, 4-TN
with open('Save_testing_results(Siam442)(prec0.4737,reca0.7391,F10.5774).txt', 'w') as f:
    for item in outs:
        f.write("%d\n" % item)

f.close()

In [ ]:
# Testing
from torchvision.transforms import ToTensor
Config.test_txt="./data/patches/testing_new_updated3.txt"   # testing_new  #validation_new_updated3
TP=0
TN=0
FP=0
FN=0

fp = open(Config.test_txt, "r")
lines= fp.readlines()

file4 = open('save_results.txt', 'w')

for i in range(4000,4500):   #  6319  1589   979  800

        line=lines[i]
        line.strip('\n')
        img_list= line.split()  #img_list has 3 elements
        img0 = Image.open(cwd+'/data/'+img_list[0])
        img1 = Image.open(cwd+'/data/'+img_list[1])
        
        img0 = Image.open(cwd+'/data/'+img_list[0])
        img1 = Image.open(cwd+'/data/'+img_list[1])

        x0 = ToTensor()(img0).unsqueeze(0)  # unsqueeze to add artificial first dimension
        x1 = ToTensor()(img1).unsqueeze(0)
        output1,output2 = net(Variable(x0).cuda(),Variable(x1).cuda())
        euclidean_distance = F.pairwise_distance(output1, output2)
        
        pred = euclidean_distance.cpu().data.numpy()[0][0]
        
        la = img_list[2]
        label = np.array(la[0][0]).astype(float)
        if (label==1.0) and (pred>0.5):
            TP = TP + 1
            file4.write("TP  %s  %f\n" % (img_list[0],pred))
            if TP<20:
                print('TP:',label,'   ',pred)
                print(img_list)
                concatenated = torch.cat((x0,x1),0)
                imshow(torchvision.utils.make_grid(concatenated))
                
        elif (label==0.0) and (pred<0.5):
            TN = TN + 1
            file4.write("TN  %s  %f\n" % (img_list[0],pred))
            if TN<0:
                print('TN:',label,'   ',pred)
                print(img_list)
                concatenated = torch.cat((x0,x1),0)
                imshow(torchvision.utils.make_grid(concatenated))
        elif (label==1.0) and (pred<= 0.5):  #0.3
            FN = FN+1
            file4.write("FN  %s  %f\n" % (img_list[0],pred))
            if FN<0:
                print('FN:',label,'   ',pred)
                print(img_list)
                concatenated = torch.cat((x0,x1),0)
                imshow(torchvision.utils.make_grid(concatenated))
        elif (label==0.0) and (pred>=0.5):   #0.8
            FP = FP+1
            file4.write("FP  %s  %f\n" % (img_list[0],pred))
            if FP<30:
                print('FP:',label,'   ',pred)
                print(img_list)
                concatenated = torch.cat((x0,x1),0)
                imshow(torchvision.utils.make_grid(concatenated))
        
        if (i%10==0):
                print('i=',i)
        
print('TP=',TP, '  TN=',TN, '  FN=', FN, '  FP=', FP)
          
# torch.from_numpy(np.array([int(img_list[2])],dtype=np.float32))

In [ ]:
print('TP=',TP, '  TN=',TN, '  FN=', FN, '  FP=', FP)
correctness = TP/(TP+FP)
completeness = TP/(TP+FN)
all_accu = (TP+TN)/(TP+TN+FN+FP)
print('correctness = ',correctness)
print('completeness = ',completeness)
print('all_accu = ',all_accu)